# Segmenting and Clustering Neighborhoods in Toronto
Course: Applied Data Science Capstone (Week 3)

We start with importing the relevant libraries.

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

# to scrape the table from the page
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
from urllib.request import urlopen

Libraries imported.


## Download and explore the data set
Here we will use BeautifulSoup to get the table.

In [7]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = urlopen(url)
soup = BeautifulSoup(html, 'html.parser')

In [8]:
#extract all tables
tables = soup.find_all('table')

After inspecting the results we find that 3 tables were extracted from the page. The first table is the one we want. And in the table, each row is under `<tr>` tag and each cell is under `<td>` tag. Below we extract the value from each cell and store them in three lists: PostalCode, Borough, and Neighborhood. In the process, boroughs that are __"Not assigned"__ are dropped. And cells that have a borough but __Not assigned__ neighborhood would use the borough value as the neighborhood.

In [27]:
#Arrays to hold extracted data
PostalCode = []
Borough = []
Neighborhood = []
table = tables[0]

rows = table.find_all('tr')
for row in rows:
    cells = row.find_all('td') # this would return empty for the header row
    
    if len(cells)>1:
        # extract the text of cells and strip the '\n' at the end
        postal = cells[0].text.strip('\n')
        borough = cells[1].text.strip('\n')
        neighborhood = cells[2].text.strip('\n')
        
        # skip cells with an unassigned borough
        if borough == 'Not assigned':
            continue
        
        PostalCode.append(postal)
        Borough.append(borough)
        if neighborhood == 'Not assigned':
            neighborhood = borough
        Neighborhood.append(neighborhood)
        

Once we have the lists, we can construct a dataframe. We also print out the first 10 rows of the dataframe so we can check if things worked.

In [31]:
#Construct dataframe
d = {'PostalCode':PostalCode, 'Borough':Borough, 'Neighborhood':Neighborhood}
df = pd.DataFrame(d)
df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [30]:
print('There are {} rows of {} variables in the dataframe'.format(df.shape[0],df.shape[1]))

There are 103 rows of 3 variables in the dataframe
